In [297]:
!pip install torch

In [298]:
!pip install gensim

In [299]:
import nltk
import numpy as np
from numpy import dot
from numpy.linalg import norm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import math
import time
import random
from nltk.corpus import reuters
from collections import Counter
nltk.download('reuters')
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("reuters")

[nltk_data] Downloading package reuters to /Users/daki/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /Users/daki/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/daki/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package reuters to /Users/daki/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


True

# Corpus setup

In [300]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [301]:
np.random.seed(42)

In [302]:
# choosing news as suggested
reuters.categories()

['acq',
 'alum',
 'barley',
 'bop',
 'carcass',
 'castor-oil',
 'cocoa',
 'coconut',
 'coconut-oil',
 'coffee',
 'copper',
 'copra-cake',
 'corn',
 'cotton',
 'cotton-oil',
 'cpi',
 'cpu',
 'crude',
 'dfl',
 'dlr',
 'dmk',
 'earn',
 'fuel',
 'gas',
 'gnp',
 'gold',
 'grain',
 'groundnut',
 'groundnut-oil',
 'heat',
 'hog',
 'housing',
 'income',
 'instal-debt',
 'interest',
 'ipi',
 'iron-steel',
 'jet',
 'jobs',
 'l-cattle',
 'lead',
 'lei',
 'lin-oil',
 'livestock',
 'lumber',
 'meal-feed',
 'money-fx',
 'money-supply',
 'naphtha',
 'nat-gas',
 'nickel',
 'nkr',
 'nzdlr',
 'oat',
 'oilseed',
 'orange',
 'palladium',
 'palm-oil',
 'palmkernel',
 'pet-chem',
 'platinum',
 'potato',
 'propane',
 'rand',
 'rape-oil',
 'rapeseed',
 'reserves',
 'retail',
 'rice',
 'rubber',
 'rye',
 'ship',
 'silver',
 'sorghum',
 'soy-meal',
 'soy-oil',
 'soybean',
 'strategic-metal',
 'sugar',
 'sun-meal',
 'sun-oil',
 'sunseed',
 'tea',
 'tin',
 'trade',
 'veg-oil',
 'wheat',
 'wpi',
 'yen',
 'zinc']

In [303]:
news_corpus = reuters.sents(categories='cocoa')
news_corpus

[['COCOA', 'EXPORTERS', 'EXPECTED', 'TO', 'LIMIT', 'SALES', 'Major', 'cocoa', 'exporters', 'are', 'likely', 'to', 'limit', 'sales', 'in', 'the', 'weeks', 'ahead', 'in', 'an', 'effort', 'to', 'boost', 'world', 'prices', ',', 'sources', 'close', 'to', 'a', 'meeting', 'of', 'the', 'Cocoa', 'Producers', 'Alliance', '(', 'CPA', ')', 'said', '.'], ['The', 'sources', 'said', 'the', 'depressed', 'world', 'market', 'had', 'been', 'one', 'of', 'the', 'main', 'topics', 'discussed', 'in', 'a', 'closed', 'door', 'meeting', 'of', 'the', '11', '-', 'member', 'CPA', 'which', 'began', 'on', 'Monday', '.'], ...]

In [304]:
from nltk.corpus import reuters

print(sorted(reuters.categories()))


['acq', 'alum', 'barley', 'bop', 'carcass', 'castor-oil', 'cocoa', 'coconut', 'coconut-oil', 'coffee', 'copper', 'copra-cake', 'corn', 'cotton', 'cotton-oil', 'cpi', 'cpu', 'crude', 'dfl', 'dlr', 'dmk', 'earn', 'fuel', 'gas', 'gnp', 'gold', 'grain', 'groundnut', 'groundnut-oil', 'heat', 'hog', 'housing', 'income', 'instal-debt', 'interest', 'ipi', 'iron-steel', 'jet', 'jobs', 'l-cattle', 'lead', 'lei', 'lin-oil', 'livestock', 'lumber', 'meal-feed', 'money-fx', 'money-supply', 'naphtha', 'nat-gas', 'nickel', 'nkr', 'nzdlr', 'oat', 'oilseed', 'orange', 'palladium', 'palm-oil', 'palmkernel', 'pet-chem', 'platinum', 'potato', 'propane', 'rand', 'rape-oil', 'rapeseed', 'reserves', 'retail', 'rice', 'rubber', 'rye', 'ship', 'silver', 'sorghum', 'soy-meal', 'soy-oil', 'soybean', 'strategic-metal', 'sugar', 'sun-meal', 'sun-oil', 'sunseed', 'tea', 'tin', 'trade', 'veg-oil', 'wheat', 'wpi', 'yen', 'zinc']


In [305]:
news_corpus = news_corpus[:300] # taking small subset for faster training

In [306]:
flatten = lambda l: [item for sublist in l for item in sublist]
news_flatten = flatten(news_corpus)
len(news_flatten)

8699

In [307]:
vocab = list(set(news_flatten))
vocab.append('<UNK>')
vocab[:5]

['early', 'provide', 'making', 'did', '77']

In [308]:
word2index = {w:i for (i, w) in enumerate(vocab)}

In [309]:
index2word = {v:k for k, v in word2index.items()} 

In [310]:
vocab_size = len(vocab)
vocab_size

1889

# Prepare training data

In [311]:
def random_batch(batch_size, corpus, window_size=2):
    # Make skip gram of custom size window
    skip_grams = []

    for sent in corpus:
        for target_index in range(window_size, len(sent) - window_size):
            target = word2index[sent[target_index]]
            context = []
            count = window_size # count of context words to pick on the left and right
            while count > 0:
                # for default window, it will get the left most and right most word
                # then the second left most and second right most word
                context.append(word2index[sent[target_index - count]])
                context.append(word2index[sent[target_index + count]])
                count -= 1

            for word in context:
                skip_grams.append([target, word])
    
    random_inputs = []
    random_labels = []
    random_index = np.random.choice(range(len(skip_grams)), batch_size, replace=False) #randomly pick without replacement
        
    for i in random_index:
        random_inputs.append([skip_grams[i][0]])  # target, e.g., 2
        random_labels.append([skip_grams[i][1]])  # context word, e.g., 3
            
    return np.array(random_inputs), np.array(random_labels)

In [312]:
#testing the method
batch_size = 2 # mini-batch size
input_batch, target_batch = random_batch(batch_size, news_corpus)

print("Input: ", input_batch)
print("Target: ", target_batch)

Input:  [[1171]
 [1111]]
Target:  [[ 667]
 [1243]]


# Skipgram

In [313]:
class Skipgram(nn.Module):
    def __init__(self, vocab_size, emb_size):
        super(Skipgram, self).__init__()

        self.embedding_center   = nn.Embedding(vocab_size, emb_size)
        self.embedding_outside  = nn.Embedding(vocab_size, emb_size)

    def forward(self, center, outside, all_vocab):
        center_embedding    = self.embedding_center(center)     # as seen in above example, size: (batch_size, 1, embedding_size)
        outside_embedding   = self.embedding_outside(outside)   # (batch_size, 1, embedding_size)
        all_vocab_embedding = self.embedding_outside(all_vocab) # (batch_size, vocab_size, embedding_size)

        numerator   = torch.exp(outside_embedding.bmm(center_embedding.transpose(1, 2)).squeeze(2))
        # (b_size, 1, emb_size) @ (b_size, emb_size, 1) = (b_size, 1, 1) -> (b_size, 1)


        denominator = all_vocab_embedding.bmm(center_embedding.transpose(1, 2)).squeeze(2)
        # (b_size, vocab_size, emb_size) @ (b_size, emb_size, 1) = (b_size, vocab_size, 1) -> (b_size, vocab_size)

        denominator_sum = torch.sum(torch.exp(denominator), 1)

        loss = -torch.mean(torch.log(numerator / denominator_sum)) # scalar

        return loss

In [314]:
batch_size     = 2 # mini-batch size
embedding_size = 2
model          = Skipgram(vocab_size, embedding_size)
model_skipgram = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

In [315]:
def test(w):
    if word2index.get(w) is not None:
        return word2index[w]
    else:
        return word2index['<UNK>']

In [316]:
def prepare_sequence(seq, word2index):
    idxs = list(map(lambda w: word2index[w] if word2index.get(w) is not None else word2index["<UNK>"], seq))
    return torch.LongTensor(idxs)

#use for the normalized term in the probability calculation
all_vocabs = prepare_sequence(list(vocab), word2index).expand(batch_size, len(vocab))  # [batch_size, voc_size]
all_vocabs.shape

torch.Size([2, 1889])

In [317]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## Training

In [318]:
# Training
num_epochs = 5000
for epoch in range(num_epochs):
    
    start = time.time()
    
    input_batch, target_batch = random_batch(batch_size, news_corpus)
    input_batch  = torch.LongTensor(input_batch)  #[batch_size, 1]
    target_batch = torch.LongTensor(target_batch) #[batch_size, 1]

    # changing to cuda
    input_batch  = input_batch.to(device)
    target_batch = target_batch.to(device)
    all_vocabs   = all_vocabs.to(device)

    optimizer.zero_grad()
    loss = model_skipgram(input_batch, target_batch, all_vocabs)
    
    loss.backward()
    optimizer.step()
    
    end = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start, end)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch: {epoch + 1} | cost: {loss:.6f} | time: {epoch_mins}m {epoch_secs}s")


Epoch: 1000 | cost: 8.070057 | time: 0m 0s
Epoch: 2000 | cost: 7.823939 | time: 0m 0s
Epoch: 3000 | cost: 7.000368 | time: 0m 0s
Epoch: 4000 | cost: 7.269746 | time: 0m 0s
Epoch: 5000 | cost: 7.897872 | time: 0m 0s


In [319]:
# Save the model
torch.save(model_skipgram.state_dict(), 'model/skipgram_model.pth')

In [320]:
import pickle
pickle.dump(model_skipgram, open('model/skipgram.pkl', 'wb'))

In [321]:
word = vocab[0]

In [322]:
#numericalization
id = word2index[word]
id

0

In [323]:
id_tensor = torch.LongTensor([id])
id_tensor = id_tensor.to(device)

In [324]:
#get the embedding by averaging
v_embed = model_skipgram.embedding_center(id_tensor)
u_embed = model_skipgram.embedding_outside(id_tensor)

v_embed, u_embed

(tensor([[ 1.0020, -0.6032]], grad_fn=<EmbeddingBackward0>),
 tensor([[ 0.0830, -0.3254]], grad_fn=<EmbeddingBackward0>))

In [325]:
#average to get the word embedding
word_embed = (v_embed + u_embed) / 2
word_embed[0][1]

tensor(-0.4643, grad_fn=<SelectBackward0>)

In [326]:
def get_embed_skip_gram(word):
    id_tensor = torch.LongTensor([word2index[word]])
    id_tensor = id_tensor.to(device)
    v_embed = model_skipgram.embedding_center(id_tensor)
    u_embed = model_skipgram.embedding_outside(id_tensor) 
    word_embed = (v_embed + u_embed) / 2 
    x, y = word_embed[0][0].item(), word_embed[0][1].item()

    return x, y

In [327]:
def cos_sim(a, b):
    cos_sim = dot(a, b)/(norm(a)*norm(b))
    return cos_sim

In [328]:
government = get_embed_skip_gram('formal')
officials = get_embed_skip_gram('almost')
administration = get_embed_skip_gram('Malaysian')

In [349]:
print(f"formal vs almost: {cos_sim(formal, almost):.4f}")
print(f"formal vs Malaysian: {cos_sim(formal, Malaysian):.4f}")
print(f"formal vs formal: {cos_sim(formal, formal):.4f}")

formal vs almost: 0.1466
formal vs Malaysian: 0.5393
formal vs formal: 1.0000


# Skipgram (Negative Sampling)

In [330]:
Z = 0.001

In [331]:
word_count = Counter(news_flatten)
num_total_words = sum([c for w, c in word_count.items()])

In [332]:
num_total_words

8699

In [333]:
unigram_table = []

for vo in vocab:
    uw = word_count[vo] / num_total_words
    uw_alpha = int((uw ** 0.75) / Z)
    unigram_table.extend([vo] * uw_alpha)

In [334]:
Counter(unigram_table)

Counter({',': 100,
         'the': 98,
         '.': 90,
         'to': 64,
         'of': 53,
         'and': 48,
         'in': 43,
         'said': 41,
         'cocoa': 38,
         'a': 34,
         'stock': 30,
         'The': 30,
         'buffer': 29,
         'for': 27,
         'is': 26,
         'be': 22,
         'from': 22,
         'ICCO': 22,
         'on': 21,
         '-': 20,
         'market': 19,
         'are': 19,
         '"': 19,
         'prices': 18,
         'at': 18,
         'by': 18,
         'as': 17,
         'will': 17,
         '000': 17,
         'manager': 16,
         'with': 16,
         'tonnes': 16,
         "'": 16,
         'that': 16,
         'which': 15,
         'Cocoa': 15,
         'not': 15,
         'International': 14,
         '1': 14,
         'COCOA': 14,
         'it': 14,
         '/': 14,
         'have': 14,
         'rules': 14,
         'they': 14,
         'were': 13,
         'delegates': 13,
         's': 13,
         '(': 

## Negative Sampling

In [335]:
def negative_sampling(targets, unigram_table, k):
    batch_size = targets.shape[0]
    neg_samples = []
    for i in range(batch_size):  #(1, k)
        target_index = targets[i].item()
        nsample      = []
        while (len(nsample) < k):
            neg = random.choice(unigram_table)
            if word2index[neg] == target_index:
                continue
            nsample.append(neg)
        neg_samples.append(prepare_sequence(nsample, word2index).reshape(1, -1))
        
    return torch.cat(neg_samples) #batch_size, k

In [336]:
batch_size = 2
x, y = random_batch(batch_size, news_corpus)
x_tensor = torch.LongTensor(x)
y_tensor = torch.LongTensor(y)
x_tensor = x_tensor.to(device)
y_tensor = y_tensor.to(device)

In [337]:
k = 5
neg_samples = negative_sampling(y_tensor, unigram_table, k)

In [338]:
y_tensor[0]

tensor([591])

In [339]:
neg_samples[0]

tensor([1279,  343, 1803, 1803,  387])

In [340]:
class SkipgramNeg(nn.Module):
    
    def __init__(self, voc_size, emb_size):
        super(SkipgramNeg, self).__init__()
        self.embedding_center  = nn.Embedding(voc_size, emb_size)
        self.embedding_outside = nn.Embedding(voc_size, emb_size)
        self.logsigmoid        = nn.LogSigmoid()
    
    def forward(self, center, outside, negative):
        #center, outside:  (bs, 1)
        #negative       :  (bs, k)
        
        center_embed   = self.embedding_center(center) #(bs, 1, emb_size)
        outside_embed  = self.embedding_outside(outside) #(bs, 1, emb_size)
        negative_embed = self.embedding_outside(negative) #(bs, k, emb_size)

        
        uovc           = outside_embed.bmm(center_embed.transpose(1, 2)).squeeze(2) #(bs, 1)
        ukvc           = -negative_embed.bmm(center_embed.transpose(1, 2)).squeeze(2) #(bs, k)
        ukvc_sum       = torch.sum(ukvc, 1).reshape(-1, 1) #(bs, 1)
        
        loss           = self.logsigmoid(uovc) + self.logsigmoid(ukvc_sum)
        
        return -torch.mean(loss)

## Testing

In [341]:
emb_size = 2
voc_size = len(vocab)
model = SkipgramNeg(voc_size, emb_size)
model_neg = model.to(device)
neg_samples = neg_samples.to(device)

In [342]:
loss = model_neg(x_tensor, y_tensor, neg_samples)
loss

tensor(2.7375, grad_fn=<NegBackward0>)

In [343]:
optimizer = optim.Adam(model_neg.parameters(), lr=0.001)

In [344]:
num_epochs = 5000

for epoch in range(num_epochs):
    
    #get batch
    input_batch, label_batch = random_batch(batch_size, news_corpus)
    input_tensor = torch.LongTensor(input_batch)
    label_tensor = torch.LongTensor(label_batch)

    #move to cuda
    input_tensor = input_tensor.to(device)
    label_tensor = label_tensor.to(device)
    
    #predict
    neg_samples = negative_sampling(label_tensor, unigram_table, k)
    neg_samples = neg_samples.to(device)

    loss = model_neg(input_tensor, label_tensor, neg_samples)
    
    #backprogate
    optimizer.zero_grad()
    loss.backward()
    
    #update alpha
    optimizer.step()
    
    #print the loss
    if (epoch + 1) % 1000 == 0:
        print(f"Epoch {epoch+1:6.0f} | Loss: {loss:2.6f}")

Epoch   1000 | Loss: 2.562646
Epoch   2000 | Loss: 1.240909
Epoch   3000 | Loss: 1.524035
Epoch   4000 | Loss: 0.893156
Epoch   5000 | Loss: 1.732999


In [345]:
# Save the model
torch.save(model_neg.state_dict(), 'model/skipgram_neg_model.pth')

In [346]:
pickle.dump(model_neg, open('model/skipgram_neg.pkl', 'wb'))

In [347]:
def get_embed_neg_sample(word):
    id_tensor = torch.LongTensor([word2index[word]])
    id_tensor = id_tensor.to(device)
    v_embed = model_neg.embedding_center(id_tensor)
    u_embed = model_neg.embedding_outside(id_tensor) 
    word_embed = (v_embed + u_embed) / 2 
    x, y = word_embed[0][0].item(), word_embed[0][1].item()

    return x, y

In [350]:
government = get_embed_neg_sample('formal')
officials = get_embed_neg_sample('almost')
administration = get_embed_neg_sample('Malaysian')

In [351]:
print(f"formal vs almost: {cos_sim(government, officials):.4f}")
print(f"formal vs Malaysian: {cos_sim(government, administration):.4f}")
print(f"formal vs formal: {cos_sim(government, government):.4f}")

formal vs almost: 0.4797
formal vs Malaysian: 0.8579
formal vs formal: 1.0000


# Glove

In [352]:
def get_skipgram(window_size = 2):
    # Make skip gram of custom size window
    skip_grams = []

    for sent in news_corpus:
        for target_index in range(window_size, len(sent) - window_size):
            target = sent[target_index]
            context = []
            count = window_size # count of context words to pick on the left and right
            while count > 0:
                # for default window, it will get the left most and right most word
                # then the second left most and second right most word
                context.append(sent[target_index - count])
                context.append(sent[target_index + count])
                count -= 1

            for word in context:
                skip_grams.append((target, word))
    return skip_grams

In [353]:
skip_grams = get_skipgram(2)
skip_grams[:8]

[('EXPECTED', 'COCOA'),
 ('EXPECTED', 'LIMIT'),
 ('EXPECTED', 'EXPORTERS'),
 ('EXPECTED', 'TO'),
 ('TO', 'EXPORTERS'),
 ('TO', 'SALES'),
 ('TO', 'EXPECTED'),
 ('TO', 'LIMIT')]

In [354]:
X_ik_skipgram = Counter(skip_grams)

In [355]:
#simply a normalized function...don't worry too much
def weighting(w_i, w_j, X_ik):
        
    #check whether the co-occurrences exist between these two words
    try:
        x_ij = X_ik[(w_i, w_j)]
    except:
        x_ij = 1  #if does not exist, set it to 1
                
    x_max = 100 #100 # fixed in paper  #cannot exceed 100 counts
    alpha = 0.75
    
    #if co-occurrence does not exceed 100, scale it based on some alpha
    if x_ij < x_max:
        result = (x_ij/x_max)**alpha  #scale it
    else:
        result = 1  #if is greater than max, set it to 1 maximum
    
    return result

In [356]:
from itertools import combinations_with_replacement

X_ik = {}  #for keeping the co-occurences
weighting_dic = {} #scaling the percentage of sampling

for bigram in combinations_with_replacement(vocab, 2):
    if X_ik_skipgram.get(bigram) is not None:  #matches 
        co_occer = X_ik_skipgram[bigram]  #get the count from what we already counted
        X_ik[bigram] = co_occer + 1 # + 1 for stability issue
        X_ik[(bigram[1],bigram[0])] = co_occer+1   #count also for the opposite
    else:
        pass
        
    weighting_dic[bigram] = weighting(bigram[0], bigram[1], X_ik)
    weighting_dic[(bigram[1], bigram[0])] = weighting(bigram[1], bigram[0], X_ik)

# print(f"{X_ik=}")
# print(f"{weighting_dic=}")

In [357]:
def random_batch(batch_size, word_sequence, skip_grams, X_ik, weighting_dic):
    
    #convert to id since our skip_grams is word, not yet id
    skip_grams_id = [(word2index[skip_gram[0]], word2index[skip_gram[1]]) for skip_gram in skip_grams]
    
    random_inputs = []
    random_labels = []
    random_coocs  = []
    random_weightings = []
    random_index = np.random.choice(range(len(skip_grams_id)), batch_size, replace=False) #randomly pick without replacement
        
    for i in random_index:
        random_inputs.append([skip_grams_id[i][0]])  # target, e.g., 2
        random_labels.append([skip_grams_id[i][1]])  # context word, e.g., 3
        
        #get cooc
        pair = skip_grams[i]
        try:
            cooc = X_ik[pair]
        except:
            cooc = 1
        random_coocs.append([math.log(cooc)])
        
        #get weighting
        weighting = weighting_dic[pair]
        random_weightings.append([weighting])
                    
    return np.array(random_inputs), np.array(random_labels), np.array(random_coocs), np.array(random_weightings)

In [358]:
#testing the method
batch_size = 2 # mini-batch size
input_batch, target_batch, cooc_batch, weighting_batch = random_batch(batch_size, news_corpus, skip_grams, X_ik, weighting_dic)

print("Input: ", input_batch)
print("Target: ", target_batch)
print("Cooc: ", cooc_batch)
print("Weighting: ", weighting_batch)


Input:  [[ 173]
 [1111]]
Target:  [[1135]
 [ 383]]
Cooc:  [[1.38629436]
 [2.94443898]]
Weighting:  [[0.08944272]
 [0.28778304]]


In [359]:
class GloVe(nn.Module):
    
    def __init__(self, vocab_size,embed_size):
        super(GloVe,self).__init__()
        self.embedding_center = nn.Embedding(vocab_size, embed_size) # center embedding
        self.embedding_outside = nn.Embedding(vocab_size, embed_size) # out embedding
        
        self.v_bias = nn.Embedding(vocab_size, 1)
        self.u_bias = nn.Embedding(vocab_size, 1)
        
    def forward(self, center_words, target_words, coocs, weighting):
        center_embeds = self.embedding_center(center_words) # [batch_size, 1, emb_size]
        target_embeds = self.embedding_outside(target_words) # [batch_size, 1, emb_size]
        
        center_bias = self.v_bias(center_words).squeeze(1)
        target_bias = self.u_bias(target_words).squeeze(1)
        
        inner_product = target_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2)
        #[batch_size, 1, emb_size] @ [batch_size, emb_size, 1] = [batch_size, 1, 1] = [batch_size, 1]
        
        #note that coocs already got log
        loss = weighting*torch.pow(inner_product +center_bias + target_bias - coocs, 2)
        
        return torch.sum(loss)

## Testing

In [360]:
batch_size     = 10 # mini-batch size
embedding_size = 2 #so we can later plot
model_glove    = GloVe(voc_size, embedding_size)
model_glove    = model_glove.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_glove.parameters(), lr=0.001)

In [361]:
num_epochs = 5000
for epoch in range(num_epochs):
    
    start = time.time()
    
    input_batch, target_batch, cooc_batch, weighting_batch = random_batch(batch_size, news_corpus, skip_grams, X_ik, weighting_dic)
    input_batch  = torch.LongTensor(input_batch)         #[batch_size, 1]
    target_batch = torch.LongTensor(target_batch)        #[batch_size, 1]
    cooc_batch   = torch.FloatTensor(cooc_batch)         #[batch_size, 1]
    weighting_batch = torch.FloatTensor(weighting_batch) #[batch_size, 1]

    # to cuda
    input_batch = input_batch.to(device)
    target_batch = target_batch.to(device)
    cooc_batch = cooc_batch.to(device)
    weighting_batch = weighting_batch.to(device)
    
    optimizer.zero_grad()
    loss = model_glove(input_batch, target_batch, cooc_batch, weighting_batch)
    
    loss.backward()
    optimizer.step()
    
    end = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start, end)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch: {epoch + 1} | cost: {loss:.6f} | time: {epoch_mins}m {epoch_secs}s")


Epoch: 1000 | cost: 4.172056 | time: 0m 0s
Epoch: 2000 | cost: 3.891329 | time: 0m 0s
Epoch: 3000 | cost: 9.241829 | time: 0m 0s
Epoch: 4000 | cost: 1.085584 | time: 0m 0s
Epoch: 5000 | cost: 4.912088 | time: 0m 0s


In [362]:
# save the model
torch.save(model_glove.state_dict(), 'model/glove_model.pth')

In [363]:
# save the model using pickle
pickle.dump(model_glove, open('model/glove.pkl', 'wb'))

In [364]:
def get_embed_glove(word):
    id_tensor = torch.LongTensor([word2index[word]])
    id_tensor = id_tensor.to(device)
    v_embed = model_glove.embedding_center(id_tensor)
    u_embed = model_glove.embedding_outside(id_tensor) 
    word_embed = (v_embed + u_embed) / 2 
    x, y = word_embed[0][0].item(), word_embed[0][1].item()

    return x, y

In [367]:
government = get_embed_glove('formal')
officials = get_embed_glove('almost')
administration = get_embed_glove('Malaysian')

In [366]:
print(f"formal vs almost: {cos_sim(government, officials):.4f}")
print(f"formal vs Malaysian: {cos_sim(government, administration):.4f}")
print(f"formal vs formal: {cos_sim(government, government):.4f}")

formal vs almost: -0.0456
formal vs Malaysian: 0.4791
formal vs formal: 1.0000


# Glove (Genism)
Source credit: https://nlp.stanford.edu/projects/glove/

In [368]:
from gensim.test.utils import datapath
from gensim.models import KeyedVectors

#you have to put this file in some python/gensim directory; just run it and it will inform where to put....
glove_file = datapath('glove.6B.100d.txt')
model_genism = KeyedVectors.load_word2vec_format(glove_file, binary=False, no_header=True)

In [369]:
# Example: Word similarity
similarity = model_genism.similarity('king', 'queen')
print(f"Similarity between 'king' and 'queen': {similarity:.4f}")

# Example: Word analogy
result = model_genism.most_similar(positive=['king', 'woman'], negative=['man'])
print("King - Man + Woman = ", result[0][0])

Similarity between 'king' and 'queen': 0.7508
King - Man + Woman =  queen


# Task-2

## Evaluating methods

In [370]:
def predict_word(word1, word2, word3, embeddings, word_to_index, index_to_word):
    # Get vectors for w1, w2, w3
    vec1 = np.array(embeddings(word1))
    vec2 = np.array(embeddings(word2))
    vec3 = np.array(embeddings(word3))

    # Calculate the predicted vector
    predicted_vec = vec1 - vec2 + vec3

    # Find the closest word by cosine similarity
    max_similarity = -1
    best_word = None
    for word, index in word_to_index.items():
        if word in [word1, word2, word3]:  # Skip the input words
            continue
        similarity = cos_sim(predicted_vec, embeddings(word))
        if similarity > max_similarity:
            max_similarity = similarity
            best_word = word

    return best_word

In [371]:
# Evaluate accuracy
def evaluate_analogies(analogy_dataset, embeddings, word_to_index):
    correct = 0
    total = 0

    for analogy in analogy_dataset:
        word1, word2, word3, word4 = analogy
        if word1 not in word_to_index or word2 not in word_to_index or word3 not in word_to_index or word4 not in word_to_index:
            continue  # Skip if any word is not in the vocabulary
        predicted_word = predict_word(word1, word2, word3, embeddings, word_to_index, {v: k for k, v in word_to_index.items()})
        if predicted_word == word4:
            correct += 1
        total += 1

    return correct / total if total > 0 else 0

## Dataset
Source credit: https://www.fit.vut.cz/person/imikolov/public/rnnlm/word-test.v1.txt

In [376]:
with open("custom_city_country_analogies.txt", "r") as file:
    lines = file.readlines()

semantic_dataset = []
for line in lines:
    # Split the line into words
    words = line.strip().split()
    if len(words) == 4:
        semantic_dataset.append([words[0], words[1], words[2], words[3]])

In [377]:
with open("custom_verb_past_analogies.txt", "r") as file:
    lines = file.readlines()

past_tense_dataset = []
for line in lines:
    words = line.strip().split()
    if len(words) == 4:
        past_tense_dataset.append([words[0], words[1], words[2], words[3]])

In [378]:
# quick check
semantic_dataset[:5]

[['Sydney', 'Australia', 'Melbourne', 'Australia'],
 ['Manchester', 'United_Kingdom', 'Liverpool', 'United_Kingdom'],
 ['Barcelona', 'Spain', 'Valencia', 'Spain'],
 ['Munich', 'Germany', 'Hamburg', 'Germany'],
 ['Milan', 'Italy', 'Naples', 'Italy']]

In [379]:
past_tense_dataset[:5]

[['build', 'built', 'grow', 'grew'],
 ['teach', 'taught', 'catch', 'caught'],
 ['send', 'sent', 'lend', 'lent'],
 ['spend', 'spent', 'bend', 'bent'],
 ['choose', 'chose', 'freeze', 'froze']]

## Syntactic Accuracy

In [380]:
accuracy = evaluate_analogies(past_tense_dataset, get_embed_skip_gram, word2index)
print(f"Syntactic Accuracy - skipgram: {accuracy * 100:.2f}%")

Syntactic Accuracy - skipgram: 0.00%


In [381]:
accuracy = evaluate_analogies(past_tense_dataset, get_embed_neg_sample, word2index)
print(f"Syntactic Accuracy - negative sample: {accuracy * 100:.2f}%")

Syntactic Accuracy - negative sample: 0.00%


In [382]:
accuracy = evaluate_analogies(past_tense_dataset, get_embed_glove, word2index)
print(f"Syntactic Accuracy - glove: {accuracy * 100:.2f}%")

Syntactic Accuracy - glove: 0.00%


In [383]:
accuracy = model_genism.evaluate_word_analogies("past_tense_lines.txt")[0]
print(f"Syntactic Accuracy - gensim: {accuracy * 100:.2f}%")

Syntactic Accuracy - gensim: 55.45%


## Semantic Accuracy 

In [384]:
accuracy = evaluate_analogies(semantic_dataset, get_embed_skip_gram, word2index)
print(f"Semantic Accuracy - skipgram: {accuracy * 100:.2f}%")

Semantic Accuracy - skipgram: 0.00%


In [385]:
accuracy = evaluate_analogies(semantic_dataset, get_embed_neg_sample, word2index)
print(f"Semantic Accuracy - negative sample: {accuracy * 100:.2f}%")

Semantic Accuracy - negative sample: 0.00%


In [386]:
accuracy = evaluate_analogies(semantic_dataset, get_embed_glove, word2index)
print(f"Semantic Accuracy - glove: {accuracy * 100:.2f}%")

Semantic Accuracy - glove: 0.00%


In [392]:
accuracy = model_genism.evaluate_word_analogies("capital.txt")[0]
print(f"Semantic Accuracy - gensim: {accuracy * 100:.2f}%")

ValueError: Missing section header before line #0 in capital.txt

| Model | Window Size | Training Loss | Training Time (sec) | Syntactic Accuracy (%) | Semantic Accuracy (%) |
|----------|----------|----------|----------|----------|----------|
| Skipgram    | 2     | 7.60     | 310     | 0.0     | 0.0   |
| Skipgram (Neg)   | 2     | 1.89     | 171     | 0.0     | 0.0     |
| Glove    | 2     | 2.33    | 34     | 0.0    | 0.0     |
| Glove (Genism)    | -     | -     | -     | 55.45     | 93.87     |

## Similarity Test
Source credit: http://alfonseca.org/eng/research/wordsim353.html

In [388]:
import pandas as pd

columns = ['Word 1', 'Word 2', 'Similarity Index']

df = pd.read_csv('custom_semantic_similarity.txt', sep='\t', header=None, names=columns)

df

,Word 1,Word 2,Similarity Index
0,teacher instructor 9.3,NaN,NaN
1,doctor physician 9.4,NaN,NaN
2,lawyer attorney 9.1,NaN,NaN
3,student pupil 8.8,NaN,NaN
4,city metropolis 8.7,NaN,NaN
...,...,...,...
80,sad unhappy 9.1,NaN,NaN
81,angry furious 8.8,NaN,NaN
82,calm peaceful 8.5,NaN,NaN
83,strong powerful 9.0,NaN,NaN


In [389]:
for index, row in df.iterrows():
    word_1 = row['Word 1']
    word_2 = row['Word 2']

    try:
        embed_1_neg_samp    = get_embed_neg_sample(word_1)
        embed_2_neg_samp    = get_embed_neg_sample(word_2)
        embed_1_skip_gram   = get_embed_skip_gram(word_1)
        embed_2_skip_gram   = get_embed_skip_gram(word_2)
        embed_1_glove       = get_embed_glove(word_1)
        embed_2_glove       = get_embed_glove(word_2)

    except KeyError:
        # Replacing missing embeddings with the embedding of '<UNK>'
        embed_1_neg_samp    = get_embed_neg_sample('<UNK>')
        embed_2_neg_samp    = get_embed_neg_sample('<UNK>')
        embed_1_skip_gram   = get_embed_skip_gram('<UNK>')
        embed_2_skip_gram   = get_embed_skip_gram('<UNK>')
        embed_1_glove       = get_embed_glove('<UNK>')
        embed_2_glove       = get_embed_glove('<UNK>')

    # Computing dot product
    df.at[index, 'dot_product_neg_samp'] = np.dot(embed_1_neg_samp, embed_2_neg_samp)
    df.at[index, 'dot_product_skip_gram'] = np.dot(embed_1_skip_gram, embed_2_skip_gram)
    df.at[index, 'dot_product_glove'] = np.dot(embed_1_glove, embed_2_glove)

In [390]:
df.head(10)

,Word 1,Word 2,Similarity Index,dot_product_neg_samp,dot_product_skip_gram,dot_product_glove
0,teacher instructor 9.3,NaN,NaN,0.23783,0.010527,0.094765
1,doctor physician 9.4,NaN,NaN,0.23783,0.010527,0.094765
2,lawyer attorney 9.1,NaN,NaN,0.23783,0.010527,0.094765
3,student pupil 8.8,NaN,NaN,0.23783,0.010527,0.094765
4,city metropolis 8.7,NaN,NaN,0.23783,0.010527,0.094765
5,car automobile 9.0,NaN,NaN,0.23783,0.010527,0.094765
6,plane aircraft 9.2,NaN,NaN,0.23783,0.010527,0.094765
7,river stream 7.9,NaN,NaN,0.23783,0.010527,0.094765
8,mountain hill 7.2,NaN,NaN,0.23783,0.010527,0.094765
9,computer laptop 8.3,NaN,NaN,0.23783,0.010527,0.094765


In [404]:
from scipy.stats import spearmanr

# Computing the Spearman correlation
correlation_pos, _ = spearmanr(df['Similarity Index'], df['dot_product_skip_gram'])
correlation_neg, _ = spearmanr(df['Similarity Index'], df['dot_product_neg_samp'])
correlation_glove, _ = spearmanr(df['Similarity Index'], df['dot_product_glove'])

/var/folders/qh/04jfxrcd49g3sj2d522qbjtw0000gn/T/ipykernel_19999/2929932877.py:4: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlation_pos, _ = spearmanr(df['Similarity Index'], df['dot_product_skip_gram'])
/var/folders/qh/04jfxrcd49g3sj2d522qbjtw0000gn/T/ipykernel_19999/2929932877.py:5: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlation_neg, _ = spearmanr(df['Similarity Index'], df['dot_product_neg_samp'])
/var/folders/qh/04jfxrcd49g3sj2d522qbjtw0000gn/T/ipykernel_19999/2929932877.py:6: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlation_glove, _ = spearmanr(df['Similarity Index'], df['dot_product_glove'])


In [399]:
print(f"Spearman Correlation Coefficient of Skipgram: {correlation_pos:.4f}")
print(f"Spearman Correlation Coefficient of Skipgram with Negative Sampling: {correlation_neg:.4f}")
print(f"Spearman Correlation Coefficient of Glove: {correlation_glove:.4f}")

Spearman Correlation Coefficient of Skipgram: nan
Spearman Correlation Coefficient of Skipgram with Negative Sampling: nan
Spearman Correlation Coefficient of Glove: nan


In [400]:
# Finding y_true based on the mean of similarity index in the df
y_true = df['Similarity Index'].mean()

print(f"y_true: {y_true:.2f}")

y_true: nan


In [403]:
correlation_coefficient = model_genism.evaluate_word_pairs('custom_semantic_similarity.txt')
print(f"Spearman Correlation Correlation coefficient of Glove (genism): {correlation_coefficient[1][0]:.2f}")

Spearman Correlation Correlation coefficient of Glove (genism): 0.50


| Model | Skipgram | NEG | GloVe | GloVe (genism) | Y_true |
|----------|----------|----------|----------|----------|----------|
| MSE    | 0.011     | 0.0385     | -0.03     | 0.5     | 5.29   |

In [405]:
def get_embed_for_corpus(model, words):
    embeddings = {}

    for word in words:
        try:
            index = word2index[word]
        except KeyError:
            index = word2index['<UNK>']

        word_tensor = torch.LongTensor([index])
        word_tensor = word_tensor.to(device)

        embed_c = model.embedding_center(word_tensor)
        embed_o = model.embedding_outside(word_tensor)
        embed = (embed_c + embed_o) / 2

        # return as dictionary with key as the word and value as the array of its embedding
        embeddings[word] = np.array([embed[0][0].item(), embed[0][1].item()])

    return embeddings

In [406]:
embed_whole_glove = get_embed_for_corpus(model_glove, vocab)
embed_whole_neg_skg = get_embed_for_corpus(model_neg, vocab)
embed_whole_skg = get_embed_for_corpus(model_glove, vocab)

In [407]:
with open('app/model_gensim.pkl', 'wb') as model_file:
    pickle.dump(model_genism, model_file)

In [408]:
with open('app/embed_skipgram_negative.pkl', 'wb') as pickle_file:
    pickle.dump(embed_whole_neg_skg, pickle_file)

print(f"File saved.")

File saved.


In [409]:
with open('app/embed_skipgram.pkl', 'wb') as pickle_file:
    pickle.dump(embed_whole_skg, pickle_file)

print(f"File saved.")

File saved.


In [410]:
with open('app/embed_glove.pkl', 'wb') as pickle_file:
    pickle.dump(embed_whole_glove, pickle_file)

print(f"File saved.")

File saved.


# Observation

As observed, for window size of 2 - skipgram had the highest (7.60) training loss while Negative skipgram had the lowest (1.89). Glove also performed much better compared to skipgram with loss of 2.33.

In case of Training time each model were trained for 5000 epoch. Skipgram took the longest time with 310s and Glove took the least amount of time with 34s. Negative sampling took 171s. 

All three models coded from scratch performed bad compared to Genism. This is because of the small corpus size and window size. All 3 models (Skipgram, Skipgram with negative sampling, Glove) had syantactic and semantic accuracy of 0%. This was expected because of the limitations of our corpus. Glove (Genism) on the other achieved Syntactic accuracy of 55.45% 
and Semantic Accuracy of 93.87%.

Furthermore, for Spearman Correlation Coefficient - Genism outperforms other models with correlation score of `0.5`. The other 3 models showed poor correlation which suggests that predicted rankings do not closely match with ground truth. So our embeddings has poor correlation with human judgement.

In conclusion, given the small corpus size, window size and embedding dimension - our 'made from scratch' models performed poorly. Given better hyperparameter tunings like embeddig dimensions, learning rate and even number of epochs, the models can be refined.